# Probabilistics Dynamical Decoupling

Ideal system:  **$e^{-iZt}$** \
Noisy implementation: **$e^{-i(Z+\lambda{X})t}$**
#### Want to prove: 
**$D=e^{-i(Z+\lambda{X})t}Ze^{-i(Z+\lambda{X})t}Z$** \
**$D_{rev}=Ze^{-i(Z+\lambda{X})t}Ze^{-i(Z+\lambda{X})t}$** \

**$||e^{-iZt} - \sum Pr(DD_{sequences})|| $** 

### Libraries and Pauli Matrices

In [4]:
# Libraries
import numpy as np
import scipy
import math
from scipy.linalg import expm, svd
import matplotlib.pyplot as plt
import scipy.optimize as spo
from scipy.integrate import quad

In [5]:
# Pauli Matrices
X, Z, I = np.matrix([[0, 1],[1, 0]]), np.matrix([[1, 0],[0, -1]]), np.matrix([[1, 0],[0, 1]])

In [6]:
def UDD_calculate_s_values(m):
    s_values = []
    prev_t = 0
    for j in range(1, m+1):
        t = np.power(np.sin(j*np.pi/(2*m)), 2)
        s_values.append(t-prev_t)
        prev_t = t
    return s_values

def error_calculation(lambda_value, m, T, areEqual, s_values):
    prev_A = I
    
    # Case 1: Periodic DD when s1 = s2 = ... = sm 
    if (areEqual == True):
        for i in np.linspace(1,m,num=m):
            # even numbers
            if (i%2 == 0): 
                A = np.dot(prev_A, expm(-1j*(Z-lambda_value*X)*T/m)) 
                #prev_A * expm(-1j*(Z-lambda_value*X)*T/m)
            # odd numbers
            else:
                A = np.dot(prev_A, expm(-1j*(Z+lambda_value*X)*T/m))
                #prev_A * expm(-1j*(Z+lambda_value*X)*T/m)
            # updating the matrix A
            prev_A = A
    
    # Case 2: Concatenated DD or Uhrig DD when s has a different values       
    else:
        i = 1
        # simulating the equation 
        for s in s_values:
            # even numbers
            if (i%2 == 0):
                A = np.dot(prev_A, expm(-1j*(Z-lambda_value*X)*s*T))
                # prev_A * expm(-1j*(Z-lambda_value*X)*s*T)   
            # odd numbers
            else:
                A = np.dot(prev_A, expm(-1j*(Z+lambda_value*X)*s*T))
                # prev_A * expm(-1j*(Z+lambda_value*X)*s*T)
            # updating variables
            #print(A)
            prev_A = A 
            i+=1 
            
    B = expm(-1j*Z*T) 
    # calculate the difference between the ideal and noisy system
    difference = A - B
    # get the maximum eigen value of the difference matrix
    eigenvalues = svd(difference, compute_uv = False)
    max_eigenvalue = np.max(eigenvalues)
    #print("The error is ", max_eigenvalue)
    return max_eigenvalue
    
def DD_sequence(lambda_value, m, T, areEqual, s_values):
    prev_A = I
    
    # Case 1: Periodic DD when s1 = s2 = ... = sm 
    if (areEqual == True):
        for i in np.linspace(1,m,num=m):
            # even numbers
            if (i%2 == 0): 
                A = np.dot(prev_A, expm(-1j*(Z-lambda_value*X)*T/m)) 
                #prev_A * expm(-1j*(Z-lambda_value*X)*T/m)
            # odd numbers
            else:
                A = np.dot(prev_A, expm(-1j*(Z+lambda_value*X)*T/m))
                #prev_A * expm(-1j*(Z+lambda_value*X)*T/m)
            # updating the matrix A
            prev_A = A
    
    # Case 2: Concatenated DD or Uhrig DD when s has a different values       
    else:
        i = 1
        # simulating the equation 
        for s in s_values:
            # even numbers
            if (i%2 == 0):
                A = np.dot(prev_A, expm(-1j*(Z-lambda_value*X)*s*T))
                # prev_A * expm(-1j*(Z-lambda_value*X)*s*T)   
            # odd numbers
            else:
                A = np.dot(prev_A, expm(-1j*(Z+lambda_value*X)*s*T))
                # prev_A * expm(-1j*(Z+lambda_value*X)*s*T)
            # updating variables
            #print(A)
            prev_A = A 
            i+=1 
    return A


### Simulation

**$||e^{-iZt} - \frac{1}{2}(UDD_{level 2} + UDD_{level 3})|| $** 

In [9]:
def probabilistic_error(DD_seq_array):
    A = np.matrix([[0+0j, 0+0j],[0+0j, 0+0j]])
    for DD in DD_seq_array:
        A += DD
        
    A = A*1/2  
    B = expm(-1j*Z*T) 
    # calculate the difference between the ideal and noisy system
    difference = A - B
    # get the maximum eigen value of the difference matrix
    eigenvalues = svd(difference, compute_uv = False)
    max_eigenvalue = np.max(eigenvalues)
    #print("The error is ", max_eigenvalue)
    return max_eigenvalue


In [10]:
# set the value of lambda and T
lambda_value = 0.01
T = 0.1


# For probabilistic error
UDD_2 = DD_sequence(lambda_value, 2, T, False, UDD_calculate_s_values(2))
UDD_3 = DD_sequence(lambda_value, 3, T, False, UDD_calculate_s_values(3))

DD_sequence = np.array([UDD_2, UDD_3])

error_avg = probabilistic_error(DD_sequence)

# Error of individual DD sequences
error_UDD_2 = error_calculation(lambda_value, 2, T, False, UDD_calculate_s_values(2))
error_UDD_3 = error_calculation(lambda_value, 3, T, False, UDD_calculate_s_values(3))

print("Probabilistic error: " + str(error_avg))
print("Error UDD(2): " + str(error_UDD_2))
print("Error UDD(3): " + str(error_UDD_3))



Probabilistic error: 2.4986981722161325e-05
Error UDD(2): 4.995834583212628e-05
Error UDD(3): 1.2492258559291002e-06
